In [36]:
import pickle
import numpy as np
import paddle
import pandas as pd
import paddle.nn as nn
from tqdm import tqdm

t_file = open('data/train_data.pkl', 'rb')
train_data = pickle.load(t_file)
v_file = open('data/eval_data.pkl', 'rb')
eval_data = pickle.load(v_file)

users_df = pd.read_csv('data/users.csv')
items_df = pd.read_csv('data/items.csv')

In [2]:
emb_scale = 1024
batch_size = 4096
len_users = len(users_df)
len_items = len(items_df)

In [6]:
class Dataset(paddle.io.Dataset):
    def __init__(self, data):
        self.feature = data[:,0:2]
        self.label = data[:,2]
        pass
    def __getitem__(self, idx):
        return self.feature[idx], self.label[idx]
    def __len__(self):
        return len(self.feature)
    pass

train_dataset = paddle.io.DataLoader(Dataset(train_data),
                               batch_size=batch_size,
                               drop_last=True,
                               shuffle=True)
eval_dataset = paddle.io.DataLoader(Dataset(eval_data),
                               batch_size=batch_size,
                               drop_last=True,
                               shuffle=True)                               

In [8]:
class DSSM(nn.Layer):
    def __init__(self):
        super(DSSM,self).__init__()
        self.users_emb = nn.Embedding(len_users+1, emb_scale)
        self.items_emb = nn.Embedding(len_items+1, emb_scale)
        self.users_fc1 = nn.Linear(emb_scale, 512)
        self.users_fc2 = nn.Linear(512, 300)
        self.users_fc3 = nn.Linear(300, 300)
        self.users_fc4 = nn.Linear(300, 128)
        self.items_fc1 = nn.Linear(emb_scale, 512)
        self.items_fc2 = nn.Linear(512, 300)
        self.items_fc3 = nn.Linear(300, 300)
        self.items_fc4 = nn.Linear(300, 128)
        pass
    def forward(self, input):
        # user tower
        user = self.users_emb(input[:,0])
        user = self.users_fc1(user)
        user = nn.functional.relu(user)
        user = self.users_fc2(user)
        user = nn.functional.relu(user)
        user = self.users_fc3(user)
        user = nn.functional.relu(user)
        user = self.users_fc4(user)
        # item tower
        item = self.items_emb(input[:,1])
        item = self.items_fc1(item)
        item = nn.functional.relu(item)
        item = self.items_fc2(item)
        item = nn.functional.relu(item)
        item = self.items_fc3(item)
        item = nn.functional.relu(item)
        item = self.items_fc4(item)
        # similarity
        x = nn.functional.cosine_similarity(user, item, axis=1)
        x = nn.functional.sigmoid(x)
        return x
    pass
dssm = DSSM()
optim = paddle.optimizer.Adam(parameters=dssm.parameters())

In [44]:
train_loss_, train_acc_, eval_loss_, eval_acc_ = [], [], [], []

for epoch in range(1):

    train_loss, train_acc, eval_loss, eval_acc = 0, 0, 0, 0
    dssm.train()

    for batch_id, data in enumerate(tqdm(train_dataset)):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))
        loss.backward()

        optim.step()
        optim.clear_grad()
        train_loss += loss
        train_acc += acc
    
    train_loss /= (batch_id + 1)
    train_acc /= (batch_id + 1)
    train_loss_.append(train_loss)
    train_acc_.append(train_acc)
    dssm.eval()

    for batch_id, data in enumerate(verify_dataset):
        x_data = data[0]
        y_data = data[1]
        x_data = paddle.cast(x_data, dtype='int32')

        y_pred = dssm(x_data)
        loss = nn.functional.mse_loss(y_pred, paddle.cast(y_data, dtype='float32'))
        acc = paddle.static.accuracy(paddle.reshape(y_pred, (batch_size, 1)), paddle.reshape(paddle.cast(y_data, dtype='int64'), (batch_size, 1)))

        eval_loss += loss
        eval_acc += acc

    eval_loss /= (batch_id + 1)
    eval_acc /= (batch_id + 1)
    eval_loss_.append(eval_loss)
    eval_acc_.append(eval_acc)


100%|██████████| 61/61 [02:22<00:00,  2.34s/it]


NameError: name 'tran_loss_' is not defined

In [45]:
train_loss

Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.17013514])

In [46]:
train_acc

Tensor(shape=[1], dtype=float32, place=Place(cpu), stop_gradient=False,
       [0.50306976])